<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-17 14:53:53
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  10.00 K
Current:  1.38 C
-------------------
Today PnL: 22.57 K (0.16%)
Current PnL: -12.69 L (-8.84%)
CY Booked + Current PnL: -4.61 L (-3.21%)
-------------------
Total profit:  4.20 L
Total loss:  -16.88 L
-------------------
Total Booked + Current PnL: 22.37 L (19.2%)
Total Booked PnL: 35.06 L (30.08%)
Curr Year Booked PnL: 8.08 L (5.83%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 70.98 L (51.26%)
Deployed:  1.17 C
Current:  1.38 C
CAGR/XIRR %: 11.48%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-228.48,H-SC,9.65,92635.0,-2681.0,2723.0,-1.07,-2.81,2.94,0.04,162.0,-0.98,0.67,82.71,OX40N,NTT,MISC
70,SBIN,760.30,863.00,-9.61,M-LC,4.19,219910.0,18430.0,8774.0,-0.22,9.15,3.99,13.51,61.0,2.10,1.59,22.90,XY25,NTT,BANKS
86,UJJIVANSFB,52.77,53.00,52.86,M-SC,24.72,132489.0,-9990.0,10612.0,-1.03,-7.01,8.01,0.44,251.0,-0.94,0.96,56.37,OX40N,NTT,BANKS
17,BRITANNIA,5190.70,6446.05,16.13,H-LC,9.24,104157.0,10724.0,11874.0,0.02,11.48,11.40,24.18,31.0,0.90,0.75,26.48,XY25,ATH,FMCG
39,HINDALCO,651.95,761.55,-21.55,H-LC,4.41,107944.0,3632.0,13903.0,1.18,3.48,12.88,16.81,9.0,0.26,0.78,20.09,X5K,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,CAMPUS,294.86,393.00,-19.88,M-SC,1.16,167002.0,-773.0,56614.0,1.89,-0.46,33.90,33.28,210.0,-0.01,1.21,29.30,XY24,NTT,FOOTWEAR
3,ALKYLAMINE,2347.98,4546.37,7.94,H-SC,20.40,100457.0,-506.0,95042.0,1.16,-0.50,94.61,93.63,150.0,-0.01,0.73,49.63,SR,ATH,CHEMICALS
12,BANKINDIA,116.91,190.00,-32.52,H-MC,10.02,178562.0,-1246.0,113655.0,-1.63,-0.69,63.65,62.52,93.0,-0.01,1.29,33.16,XR,NTT,BANKS
61,PGHH,13388.00,18062.58,-20.22,H-MC,7.86,201210.0,390.0,69719.0,1.59,0.19,34.65,34.92,92.0,0.01,1.45,4.88,X40,ATH,FMCG
59,NATIONALUM,189.63,247.44,-50.15,H-MC,5.25,100345.0,-159.0,30796.0,0.70,-0.16,30.69,30.49,83.0,-0.01,0.73,32.20,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.00,-22.84,H-LC,0.26,203048.0,-1452.0,42863.0,0.43,-0.71,21.11,20.25,10.0,-0.03,1.47,7.03,X40N,BTT,PHARMA
40,HINDUNILVR,2413.81,2723.00,-10.06,H-LC,1.10,281882.0,11535.0,23086.0,-0.03,4.27,8.19,12.81,16.0,0.50,2.04,17.89,XY25,NTT,FMCG
37,HCLTECH,1647.60,1972.35,6.26,H-LC,2.37,94330.0,-6174.0,25978.0,-1.05,-6.14,27.54,19.71,12.0,-0.24,0.68,13.84,X40,ATH,IT
10,BAJAJHFL,122.26,152.00,-3.02,H-LC,2.97,201195.0,689.0,48086.0,-0.55,0.34,23.90,24.33,29.0,0.01,1.45,16.02,X40N,BTT,FINANCE
41,HINDZINC,471.44,730.22,15.78,H-LC,3.34,189312.0,-15764.0,128335.0,-0.51,-7.69,67.79,54.89,2.0,-0.12,1.37,14.56,X5K,ATH,METALS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,SONACOMS,578.05,1006.00,-33.95,M-SC,15.03,85181.0,-15978.0,90871.0,6.97,-15.79,106.68,74.03,202.0,-0.18,0.62,20.11,AR,BTT,AUTO
8,AWL,325.39,485.00,-66.94,H-MC,7.95,229845.0,-38602.0,170269.0,6.19,-14.38,74.08,49.05,82.0,-0.23,1.66,18.18,XY24,NTT,FMCG
64,RELAXO,902.64,1176.00,-27.28,H-SC,18.50,65773.0,-49765.0,84755.0,3.85,-43.07,128.86,30.28,131.0,-0.59,0.48,28.49,X40N,NTT,FOOTWEAR
94,WHIRLPOOL,1219.98,2270.00,-37.29,M-SC,12.13,105405.0,13906.0,64845.0,2.27,15.20,61.52,86.07,205.0,0.21,0.76,52.48,XR,NTT,DURABLES
91,VBL,492.64,672.28,-13.02,H-LC,14.96,264104.0,-936.0,97586.0,2.25,-0.35,36.95,36.46,3.0,-0.01,1.91,12.79,X40N,ATH,FMCG


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,1.69,19511.0,-30082.0,1284.0,-6.17,-60.66,6.58,-58.07,201.0,-23.43,0.14,30.84,XY24,ATH,MISC
56,LTIM,5564.16,7230.20,-2.47,H-LC,3.71,187722.0,-12588.0,72573.0,-2.11,-6.28,38.66,29.94,28.0,-0.17,1.36,29.69,X200,ATH,IT
12,BANKINDIA,116.91,190.00,-32.52,H-MC,10.02,178562.0,-1246.0,113655.0,-1.63,-0.69,63.65,62.52,93.0,-0.01,1.29,33.16,XR,NTT,BANKS
80,TATAELXSI,7332.28,7332.00,-17.14,H-MC,0.77,86954.0,-15698.0,15695.0,-1.58,-15.29,18.05,-0.00,54.0,-1.00,0.63,32.18,OX40N,NTT,IT
90,VALIANTORG,512.64,838.00,-348.91,H-SC,19.39,105820.0,-27466.0,112063.0,-1.56,-20.61,105.90,63.47,151.0,-0.25,0.76,77.38,XR,NTT,CHEMICALS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-228.48,H-SC,9.65,92635.0,-2681.0,2723.0,-1.07,-2.81,2.94,0.04,162.0,-0.98,0.67,82.71,OX40N,NTT,MISC
86,UJJIVANSFB,52.77,53.0,52.86,M-SC,24.72,132489.0,-9990.0,10612.0,-1.03,-7.01,8.01,0.44,251.0,-0.94,0.96,56.37,OX40N,NTT,BANKS
78,SYMPHONY,1306.42,1306.0,-27.47,M-SC,12.79,154698.0,-16443.0,16383.0,-0.44,-9.61,10.59,-0.03,195.0,-1.00,1.12,11.19,OX40N,NTT,DURABLES
21,CERA,7989.07,8422.0,-11.22,H-SC,1.98,127053.0,-16750.0,24547.0,1.92,-11.65,19.32,5.42,155.0,-0.68,0.92,40.21,OX40N,NTT,CERAMICS
46,INDIGOPNTS,1407.73,1408.0,127.68,H-SC,10.22,149172.0,-25387.0,25419.0,0.22,-14.54,17.04,0.02,129.0,-1.00,1.08,29.35,OX40N,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,0.34,77185.0,42736.0,2231.0,0.04,124.06,2.89,130.54,1.0,19.16,0.56,18.03,AR,ATH,ELECTRICAL
41,HINDZINC,471.44,730.22,15.78,H-LC,3.34,189312.0,-15764.0,128335.0,-0.51,-7.69,67.79,54.89,2.0,-0.12,1.37,14.56,X5K,ATH,METALS
91,VBL,492.64,672.28,-13.02,H-LC,14.96,264104.0,-936.0,97586.0,2.25,-0.35,36.95,36.46,3.0,-0.01,1.91,12.79,X40N,ATH,FMCG
82,TCS,3794.03,4998.00,-22.88,H-LC,10.74,250942.0,-44992.0,138896.0,-0.49,-15.20,55.35,31.73,4.0,-0.32,1.81,0.78,X40,BTT,IT
48,INFY,1461.46,2275.00,-11.34,H-LC,7.47,284001.0,22400.0,123228.0,-1.32,8.56,43.39,55.67,6.0,0.18,2.05,15.19,X40,BTT,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,249.44,NaN,NaN,0.90,54531.0,13276.0,98.0,-0.18,32.18,0.18,32.41,196.0,135.47,0.39,28.38,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,21.8,H-SC,6.15,156182.0,29982.0,16196.0,0.60,23.76,10.37,36.59,153.0,1.85,1.13,39.25,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3176.70,NaN,NaN,0.34,77185.0,42736.0,2231.0,0.04,124.06,2.89,130.54,1.0,19.16,0.56,18.03,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SAIL,130.64,228.00,91.93,M-MC,11.77,229818.0,4856.0,162803.0,-0.09,2.16,70.84,74.53,194.0,0.03,1.66,33.43,XY24,BTT,STEEL
53,KPIGREEN,497.21,731.64,35.49,H-SC,9.78,138739.0,13442.0,45631.0,-1.06,10.73,32.89,47.15,141.0,0.29,1.00,73.57,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,21.80,H-SC,6.15,156182.0,29982.0,16196.0,0.60,23.76,10.37,36.59,153.0,1.85,1.13,39.25,XY24,NTT,HEALTHCARE
20,CAMS,3643.00,5226.82,5.65,H-SC,3.17,118765.0,16761.0,27589.0,0.82,16.43,23.23,43.48,122.0,0.61,0.86,35.97,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2337.55,0.40,H-SC,12.40,188400.0,23730.0,45348.0,1.26,14.41,24.07,41.95,119.0,0.52,1.36,22.95,X40N,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,0.34,77185.0,42736.0,2231.0,0.04,124.06,2.89,130.54,1.0,19.16,0.56,18.03,AR,ATH,ELECTRICAL
57,MASFIN,326.60,399.50,-9.67,H-SC,13.99,102855.0,4875.0,16992.0,-0.97,4.98,16.52,22.32,146.0,0.29,0.74,49.18,XR,ATH,FINANCE
53,KPIGREEN,497.21,731.64,35.49,H-SC,9.78,138739.0,13442.0,45631.0,-1.06,10.73,32.89,47.15,141.0,0.29,1.00,73.57,MH,ATH,POWER
94,WHIRLPOOL,1219.98,2270.00,-37.29,M-SC,12.13,105405.0,13906.0,64845.0,2.27,15.20,61.52,86.07,205.0,0.21,0.76,52.48,XR,NTT,DURABLES
45,INDIAMART,2327.09,4911.36,-46.40,H-SC,10.92,139406.0,16070.0,120893.0,-1.08,13.03,86.72,111.05,118.0,0.13,1.01,36.59,AR,ATH,MISC


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,1.69,19511.0,-30082.0,1284.0,-6.17,-60.66,6.58,-58.07,201.0,-23.43,0.14,30.84,XY24,ATH,MISC
28,DIGITIDE,188.38,249.44,NaN,NaN,0.90,54531.0,13276.0,98.0,-0.18,32.18,0.18,32.41,196.0,135.47,0.39,28.38,XY24,ATH,IT
74,SIS,477.00,477.00,2322.80,M-SC,4.36,65178.0,-15912.0,15910.0,0.12,-19.62,24.41,0.00,237.0,-1.00,0.47,32.09,OX40N,NTT,MISC
27,DEN,52.79,75.00,135.75,L-SC,6.17,65936.0,-18528.0,54061.0,0.61,-21.94,81.99,42.07,266.0,-0.34,0.48,40.03,AR,NTT,ENTERTAINMENT
24,COFFEEDAY,59.14,80.00,-47.69,L-SC,20.17,77088.0,-36461.0,76510.0,0.48,-32.11,99.25,35.27,268.0,-0.48,0.56,88.67,XR,NTT,HOTELS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,1.69,19511.0,-30082.0,1284.0,-6.17,-60.66,6.58,-58.07,201.0,-23.43,0.14,30.84,XY24,ATH,MISC
64,RELAXO,902.64,1176.00,-27.28,H-SC,18.50,65773.0,-49765.0,84755.0,3.85,-43.07,128.86,30.28,131.0,-0.59,0.48,28.49,X40N,NTT,FOOTWEAR
3,ALKYLAMINE,2347.98,4546.37,7.94,H-SC,20.40,100457.0,-506.0,95042.0,1.16,-0.50,94.61,93.63,150.0,-0.01,0.73,49.63,SR,ATH,CHEMICALS
24,COFFEEDAY,59.14,80.00,-47.69,L-SC,20.17,77088.0,-36461.0,76510.0,0.48,-32.11,99.25,35.27,268.0,-0.48,0.56,88.67,XR,NTT,HOTELS
27,DEN,52.79,75.00,135.75,L-SC,6.17,65936.0,-18528.0,54061.0,0.61,-21.94,81.99,42.07,266.0,-0.34,0.48,40.03,AR,NTT,ENTERTAINMENT


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.38
1,25,40.78
2,50,69.85


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.13
LC    34.79
MC    21.99
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.79
X40N     14.36
X40      12.44
XY25     10.91
XR        9.36
OX40N     8.53
AR        8.16
X5K       2.15
MH        1.73
X200      1.36
SR        1.21
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.99
H-SC    25.68
H-MC    18.93
M-SC    14.93
M-LC     5.77
M-MC     2.70
L-SC     1.52
L-LC     1.03
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.16,0.22,33.16
IT,11.76,-11.54,66.52
FINANCE,8.52,-10.65,50.80
BANKS,7.77,-6.12,49.41
MISC,5.96,-16.57,71.40
PAINTS,5.55,-13.61,36.00
ELECTRICAL,5.30,-0.41,47.99
HEALTHCARE,4.37,0.18,26.33
AUTO,4.33,-12.95,62.62


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2588677.0,25
AR,1023409.0,10
XR,998325.0,13
X40,731708.0,10
X40N,565237.0,12
XY25,428244.0,8
OX40N,294433.0,11
SR,176730.0,2
X5K,142238.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2273081.0,26
M-SC,1566953.0,20
H-LC,1312087.0,21
H-MC,1086177.0,15
M-MC,303471.0,2
M-LC,263460.0,4
L-SC,189451.0,3
L-MC,53217.0,1
L-LC,46491.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,884989.0,7
M-SC,XY24,772489.0,7
H-SC,AR,517831.0,3
H-LC,X40,472929.0,6
H-MC,XY24,422048.0,4
H-SC,XR,327872.0,4
M-MC,XY24,303471.0,2
H-LC,X40N,293200.0,6
M-SC,XR,269610.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
